# Sellke construction of the SIR model
Sean L. Wu (@slwu89), 2021-11-30

## Introduction

The Sellke construction is an alternative stochastic process to the standard stochastic epidemic
process that can be used to sample the final size distribution of an SIR epidemic in a finite population,
originally described in [On the asymptotic distribution of the size of a stochastic epidemic](https://doi.org/10.2307/3213811).

Rather than sampling next events and times under a race condition to update the system as in
standard stochastic epidemics, the Sellke construction samples at once a "threshold" value
for each susceptible individual. The "infection pressure" generated by the infectious population
is tracked, and when it exceeds the threshold for a given susceptible person, they become infected.

This construction is similar to a [random time change](http://bactra.org/notebooks/random-time-changes.html) such
that the thresholds are independent and identically distributed exponential random variates with a mean of 1,
and infection pressure is the cumulative hazard $\beta \, I/N$.

## Libraries

In [ ]:
using Distributions
using Random
using BenchmarkTools
using Plots

## Sellke model

Code to draw a final epidemic size from a standard continuous-time Markov SIR model
is below. The Sellke construction can handle non-exponentially distributed infectious
periods, which simply require the the infectious durations to be sampled from the appropriate
distribution. Another nice feature of the Sellke epidemic model is that parameters can
easily be changed without needing to draw new random numbers.

Because the construction samples the final epidemic size, we do not specify the time domain.

In [ ]:
function sellke(u0, p)
    (S, I, R) = u0
    N = S+I+R
    (β, c, γ) = p
    λ = β*c/N

    Q = rand(Exponential(), S)
    sort!(Q)

    T0 = rand(Exponential(1/γ), I)
    T = rand(Exponential(1/γ), S)

    ST0 = sum(T0)
    Y = [ST0; ST0 .+ cumsum(T[1:end-1])]

    Z = findfirst(Q .> Y*λ)

    if Z === nothing
        return S+I # entire population infected
    else
        return Z+I-1
    end
end

## Initial conditions

In [ ]:
u0 = [100,10,0]; # S,I,R

## Parameter values

In [ ]:
p = [0.05,10.0,0.25]; # β,c,γ

## Random number seed

We set a random number seed for reproducibility.

In [ ]:
Random.seed!(1234);

## Running the model

We will sample a large number of final epidemic sizes and plot
a histogram.

In [ ]:
out_sellke = map(x -> sellke(u0, p), 1:1e3)

## Plotting

We plot a histogram of final epidemic sizes.

In [ ]:
histogram(out_sellke, bins=20, xlabel = "Final Epidemic Size", ylabel = "Frequency", legend = false)

## Benchmarking

In [ ]:
@benchmark sellke(u0, p)